In [30]:
user_input = 'I want to take a software development course, give me some recommendations'
rag_num_return = 10

rag_model_path = '../models/gte-base-en-v1.5'
rerank_model_path = "../models/gte-multilingual-reranker-base"
llm_model_name = "../models/Qwen2.5-0.5B-Instruct"

# RAG

In [31]:
from pymilvus import MilvusClient
from sentence_transformers import SentenceTransformer
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import AutoModelForCausalLM, AutoTokenizer

client = MilvusClient("dukies.db")


In [32]:
device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
print(f'using {device} for inferencing')

model = SentenceTransformer(rag_model_path, trust_remote_code=True)
model.to(device)

query_vectors = model.encode([user_input])

res = client.search(
    collection_name="rag_collection",  # target collection
    data=query_vectors,  # query vectors
    limit=rag_num_return,  # number of returned entities
    output_fields=["text", "subject"],  # specifies fields to be returned
)

res_list = [res[0][i]['entity']['text'] for i in range(len(res[0]))]
dist_list = [res[0][i]['distance'] for i in range(len(res[0]))]

using mps for inferencing


In [33]:
for res in res_list:
    print(res)

Course Code: ECE 651K
Course Name: Software Engineering
Description:
Teaches students about all steps of the software development lifecycle: requirements definition, design, development, testing, and maintenance. The course assumes students are skilled object-oriented programmers from prior courses, but will include a rapid introduction to Java. Students complete team-based semester-long software project which will progress through all phases of the software lifecycle. Taught at Duke Kunshan University in Kunshan, China. Prerequisite: ECE 551K.
Course Code: ECE 564
Course Name: Mobile Application Development
Description:
Explores mobile application development in the Apple Development Environment. Uses core software engineering pillars of Swift, Xcode, iOS & Xcode Cloud to learn how to create apps for Apple products. Focuses on iOS/iPhone, but Xcode also allows for exploration into the VisionPro, the Apple Watch, the iPad, and Apple TV. Real world context focused on common programming 

# Rerank

In [34]:
tokenizer = AutoTokenizer.from_pretrained(rerank_model_path)
model = AutoModelForSequenceClassification.from_pretrained(
    rerank_model_path, trust_remote_code=True,
    torch_dtype=torch.float16
).to(device)
model.eval()

pairs = [[user_input, res_list[i]] for i in range(len(res_list))]

with torch.no_grad():
    inputs = tokenizer(pairs, padding=True, truncation=True, return_tensors='pt', max_length=512).to(device)
    scores = model(**inputs, return_dict=True).logits.view(-1, ).float()

selected_responses = [res_list[i] for i, score in enumerate(scores) if score > 0]


In [35]:
def remove_empty_lines(input_string):
    return '\n'.join(line for line in input_string.splitlines() if line.strip())
selected_responses = [remove_empty_lines(s) for s in selected_responses]

rag_str = '\n\n'.join([response for response in selected_responses])


final_prompt = f'''Answer the following questions as best you can. You have refer to the following context:

----------
{rag_str}
----------

Question: {user_input}
'''
print(final_prompt)

Answer the following questions as best you can. You have refer to the following context:

----------
Course Code: ECE 651K
Course Name: Software Engineering
Description:
Teaches students about all steps of the software development lifecycle: requirements definition, design, development, testing, and maintenance. The course assumes students are skilled object-oriented programmers from prior courses, but will include a rapid introduction to Java. Students complete team-based semester-long software project which will progress through all phases of the software lifecycle. Taught at Duke Kunshan University in Kunshan, China. Prerequisite: ECE 551K.

Course Code: ECE 564
Course Name: Mobile Application Development
Description:
Explores mobile application development in the Apple Development Environment. Uses core software engineering pillars of Swift, Xcode, iOS & Xcode Cloud to learn how to create apps for Apple products. Focuses on iOS/iPhone, but Xcode also allows for exploration into the

In [36]:
model = AutoModelForCausalLM.from_pretrained(
    llm_model_name,
    torch_dtype="auto",
    device_map="auto"
).to(device)
tokenizer = AutoTokenizer.from_pretrained(llm_model_name)

messages = [
    {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
    {"role": "user", "content": final_prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(device)
with torch.no_grad():  # Disable gradient calculations for inference
    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=512,
    )
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

print(response)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Certainly! Here are some suggestions based on the provided information:

### Course Recommendation

**ECE 651K:**
- **Course Description:** This is a comprehensive course covering all stages of the software development lifecycle (requirements definition, design, development, testing, and maintenance). It assumes prior knowledge in object-oriented programming from previous courses and introduces you to Java.
  
- **Prerequisites:** While not strictly necessary, familiarity with object-oriented programming concepts and familiarity with basic Java syntax would be beneficial.

**ECE 564:**
- **Course Description:** This course focuses on mobile application development using Swift, Xcode, iOS, and Xcode Cloud. It covers creating apps for Apple products, focusing on both iOS and iPhone platforms.

- **Prerequisites:** A solid foundation in Objective-C and familiarity with Xcode are required.

**CompSci 408:**
- **Course Description:** This course aims to help develop software in teams while 